In [2]:
import os
import glob
import numpy as np
import random
import csv
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
from tqdm import tqdm
import io

# Set reproducibility seeds
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Custom dataset for loading .npy sequence data\ 
class SequenceDataset(Dataset):
    def __init__(self, file_paths, labels, n_mfcc=None):
        """
        If n_mfcc is given, we’ll assume any 1D .npy is a flattened MFCC of shape (n_mfcc * T,)
        and reshape it back to (n_mfcc, T).  Otherwise we leave 2D arrays alone.
        """
        self.file_paths = file_paths
        self.labels     = labels

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        arr  = np.load(path)
        self.n_mfcc = "MFCC" in path
        # if 1D and user told us how many MFCC coeffs, reshape
        if self.n_mfcc and arr.ndim == 1:
            T = arr.size // 20
            arr = arr.reshape(20, T)
        # Now arr is (freq_bins, time_steps)
        x = torch.tensor(arr, dtype=torch.float32).transpose(0,1)
        # => shape (time_steps, freq_bins), ready for LSTM
        y = torch.tensor(self.labels[idx], dtype=torch.float32)
        return x, y

# Collate pad variable-length sequences
def collate_fn(batch):
    sequences = [b[0] for b in batch]
    labels = torch.stack([b[1] for b in batch])
    # pad sequences to max length in batch
    lengths = [seq.size(0) for seq in sequences]
    max_len = max(lengths)
    feat_dim = sequences[0].size(1)
    padded = torch.zeros(len(sequences), max_len, feat_dim)
    for i, seq in enumerate(sequences):
        padded[i, :seq.size(0), :] = seq
    return padded, lengths, labels

# Two-layer LSTM model\ 
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, fc_dim, num_layers=2, dropout=0.5):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, fc_dim)
        self.out = nn.Linear(fc_dim, 1)
    def forward(self, x, lengths):
        # pack_padded_sequence for efficiency
        packed = nn.utils.rnn.pack_padded_sequence(x, lengths,
                                                   batch_first=True,
                                                   enforce_sorted=False)
        packed_out, (hn, cn) = self.lstm(packed)
        # use last hidden state from top layer
        last = hn[-1]
        x = torch.relu(self.fc(last))
        x = self.out(x)
        return x.squeeze(1)

# Training and evaluation routine
def train_eval_lstm(file_paths_train, labels_train,
                    file_paths_val, labels_val,
                    input_dim, hidden_dim, fc_dim,
                    batch_size=32, epochs=10, lr=1e-3,
                    device=None):
    set_seed()
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Training on device: {device}")

    # DataLoaders
    ds_train = SequenceDataset(file_paths_train, labels_train)
    dl_train = DataLoader(ds_train, batch_size=batch_size,
                          shuffle=True, num_workers=0,
                          collate_fn=collate_fn,
                          pin_memory=(device=='cuda'))
    ds_val = SequenceDataset(file_paths_val, labels_val)
    dl_val = DataLoader(ds_val, batch_size=batch_size,
                        shuffle=False, num_workers=0,
                        collate_fn=collate_fn,
                        pin_memory=(device=='cuda'))

    # Model, loss, optimizer
    model = LSTMClassifier(input_dim, hidden_dim, fc_dim).to(device)
    model.lstm.flatten_parameters()
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    print("params", sum(p.numel() for p in model.parameters()))
    buffer = io.BytesIO()
    torch.save(model.state_dict(), buffer)
    print("size in bytes", buffer.tell())
    return model, 0
    
    # Training loop
    for epoch in range(1, epochs+1):
        model.train()
        total_loss = 0.0
        for X, lengths, y in tqdm(dl_train, desc = "Training"):
            X, y = X.to(device), y.to(device)
            logits = model(X, lengths)
            loss = criterion(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch}/{epochs} - Loss: {total_loss/len(dl_train):.4f}")

    # Evaluation
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for X, lengths, y in tqdm(dl_val, desc="Val"):
            X = X.to(device)
            logits = model(X, lengths)
            batch_preds = (torch.sigmoid(logits).cpu().numpy() > 0.5).astype(int).tolist()
            preds.extend(batch_preds)
            trues.extend(y.numpy().astype(int).tolist())
    f1 = f1_score(trues, preds, average='macro')
    print(f"LSTM Macro-F1: {f1:.4f}")
    return model, f1
# Grid search
def grid_search(root_dir,
                sr,
                split_train='train',
                split_val='validation',
                batch_size=32,
                epochs=10,
                lr=1e-3,
                seeds=42):
    # collect file paths and labels\
    file_paths_train, labels_train = [], []
    for label, cls in [('unfiltered', 0), ('filtered', 1)]:
        folder = os.path.join(root_dir, str(sr), split_train, label)
        for fpath in tqdm(glob.glob(os.path.join(folder, '*.npy')), desc="Getting train files"):
            file_paths_train.append(fpath)
            labels_train.append(cls)
    file_paths_val, labels_val = [], []
    for label, cls in [('unfiltered', 0), ('filtered', 1)]:
        folder = os.path.join(root_dir, str(sr), split_val, label)
        for fpath in tqdm(glob.glob(os.path.join(folder, '*.npy')), desc="Getting labels"):
            file_paths_val.append(fpath)
            labels_val.append(cls)

    # assume input_dim from first sample 
    sample = np.load(file_paths_train[0])
    input_dim = sample.shape[0] if not "MFCC" in file_paths_train[0] else 20

    # define grid
    hidden_dims = [64, 128]
    fc_dims     = [32, 64]
    set_seed(seeds)

    results = []
    for h in hidden_dims:
        for f in fc_dims:
            model, f1 = train_eval_lstm(
                file_paths_train=file_paths_train, 
                labels_train=labels_train,
                file_paths_val=file_paths_val, 
                labels_val=labels_val,
                input_dim=input_dim,
                hidden_dim=h,
                fc_dim=f,
                batch_size=batch_size,
                epochs=epochs,
                lr=lr,
                device='cuda' if torch.cuda.is_available() else 'cpu'
            )
            print(f"sr={sr}, hidden={h}, fc={f} => f1={f1:.4f}")
            results.append({'sample_rate': sr,
                            'hidden_dim': h,
                            'fc_dim': f,
                            'f1_score': f1})
    # # save to CSV\    
    # timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    # fname = f"lstm_grid_{os.path.basename(root_dir)}_{sr}_{timestamp}.csv"
    # with open(fname, 'w', newline='') as cf:
    #     writer = csv.DictWriter(cf, fieldnames=['sample_rate','hidden_dim','fc_dim','f1_score'])
    #     writer.writeheader()
    #     writer.writerows(results)
    # print(f"Grid results saved to {fname}")

if __name__ == '__main__':
    print(torch.cuda.is_available())   # should be True
    print(torch.cuda.device_count())   # how many GPUs
    print(torch.cuda.get_device_name(0))
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Running on:", device)
    root = r"D:\Aliasing3\Processed_Files\DS_ZEN"
    grid_search(root, sr=1000)
    grid_search(root, sr=2000)
    grid_search(root, sr=4000)
    grid_search(root, sr=8000)
    grid_search(root, sr=16000)
    grid_search(root, sr=22000)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


True
1
NVIDIA GeForce RTX 3050 Laptop GPU
Running on: cuda:0


Getting labels: 100%|████████████████████████████████████████████████████████████████████████| 555/555 [00:00<?, ?it/s]


Training on device: cuda
params 118081
size in bytes 475084
sr=1000, hidden=64, fc=32 => f1=0.0000
Training on device: cuda
params 120193
size in bytes 483532
sr=1000, hidden=64, fc=64 => f1=0.0000
Training on device: cuda
params 334401
size in bytes 1340364
sr=1000, hidden=128, fc=32 => f1=0.0000
Training on device: cuda
params 338561
size in bytes 1357004
sr=1000, hidden=128, fc=64 => f1=0.0000


Getting labels: 100%|████████████████████████████████████████████████████████████████████████| 564/564 [00:00<?, ?it/s]


...

Training on device: cuda
params 118081
size in bytes 475084
sr=8000, hidden=64, fc=32 => f1=0.0000
Training on device: cuda
params 120193
size in bytes 483532
sr=8000, hidden=64, fc=64 => f1=0.0000
Training on device: cuda
params 334401
size in bytes 1340364
sr=8000, hidden=128, fc=32 => f1=0.0000
Training on device: cuda
params 338561
size in bytes 1357004
sr=8000, hidden=128, fc=64 => f1=0.0000


Getting labels: 100%|███████████████████████████████████████████████████████████| 580/580 [00:00<00:00, 1149124.38it/s]


Training on device: cuda
params 118081
size in bytes 475084
sr=16000, hidden=64, fc=32 => f1=0.0000
Training on device: cuda
params 120193
size in bytes 483532
sr=16000, hidden=64, fc=64 => f1=0.0000
Training on device: cuda
params 334401
size in bytes 1340364
sr=16000, hidden=128, fc=32 => f1=0.0000
Training on device: cuda
params 338561
size in bytes 1357004
sr=16000, hidden=128, fc=64 => f1=0.0000


Getting labels: 100%|████████████████████████████████████████████████████████████| 522/522 [00:00<00:00, 518575.72it/s]


Training on device: cuda
params 118081
size in bytes 475084
sr=22000, hidden=64, fc=32 => f1=0.0000
Training on device: cuda
params 120193
size in bytes 483532
sr=22000, hidden=64, fc=64 => f1=0.0000
Training on device: cuda
params 334401
size in bytes 1340364
sr=22000, hidden=128, fc=32 => f1=0.0000
Training on device: cuda
params 338561
size in bytes 1357004
sr=22000, hidden=128, fc=64 => f1=0.0000


In [5]:
if __name__ == '__main__':
    print(torch.cuda.is_available())   # should be True
    print(torch.cuda.device_count())   # how many GPUs
    print(torch.cuda.get_device_name(0))
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Running on:", device)
    root = r"D:\Aliasing3\Processed_Files\DS_ESC_MFCC"
    grid_search(root, sr=1000)
    grid_search(root, sr=2000)
    grid_search(root, sr=4000)
    grid_search(root, sr=8000)
    grid_search(root, sr=16000)
    grid_search(root, sr=22000)

True
1
NVIDIA GeForce RTX 3050 Laptop GPU
Running on: cuda:0


Getting labels: 100%|████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 214143.83it/s]


Training on device: cuda


Training: 100%|████████████████████████████████████████████████████████████████████████| 95/95 [00:54<00:00,  1.73it/s]


Epoch 1/10 - Loss: 0.6894


...

Epoch 9/10 - Loss: 0.6462


Training: 100%|████████████████████████████████████████████████████████████████████████| 95/95 [00:02<00:00, 37.60it/s]


Epoch 10/10 - Loss: 0.6422


Val: 100%|█████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 50.58it/s]

LSTM Macro-F1: 0.6124
sr=22000, hidden=128, fc=64 => f1=0.6124
Grid results saved to lstm_grid_DS_ESC_MFCC_22000_20250720_204640.csv


In [3]:
if __name__ == '__main__':
    print(torch.cuda.is_available())   # should be True
    print(torch.cuda.device_count())   # how many GPUs
    print(torch.cuda.get_device_name(0))
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Running on:", device)
    root = r"D:\Aliasing3\Processed_Files\DS_ZEN"
    grid_search(root, sr=1000)
    grid_search(root, sr=2000)
    grid_search(root, sr=4000)
    grid_search(root, sr=8000)
    grid_search(root, sr=16000)
    grid_search(root, sr=22000)

True
1
NVIDIA GeForce RTX 3050 Laptop GPU
Running on: cuda:0


Getting labels: 100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


Training on device: cuda


Training:   0%|                                                                                  | 0/1 [00:00<?, ?it/s]C:\Users\jta20\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:1135: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\RNN.cpp:1410.)
  result = _VF.lstm(
Training:   0%|                                                                                  | 0/1 [00:00<?, ?it/s]


RuntimeError: shape '[65792, 1]' is invalid for input of size 1280